# TME 7 : Mini-projet Detection de motifs


## Partie C : Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons implémenter deux algorithmes randomisés pour la recherche de motifs : Greedy Profile Motif Search et random Projections.


1\. Nous allons réutiliser les fonctions du TME6 pour générer t séquences artificielles de taille n, et implanter dans chaque séquence un motif de taille k à des positions aléatoires avec v substitutions choisies aléatoirement.

In [28]:
import random
import numpy as np
import copy

nuc = ('A', 'C', 'G', 'T')

k=7 #taille de motif
v=4 #nb de positions variable dans le motif
t=20 #nb de sequences
n=50 #longuer des sequence

def generateRandomSequence(n, I):
    seq_tmp = ""
    
    if I == True:
        for i in range(n):
            seq_tmp = seq_tmp + random.choice(nuc)
    else:
        for i in range(n):
            seq_tmp = seq_tmp + random.choice(nuc).lower()

    return seq_tmp
    

def generateRandomSequences(n, t):
    sequences = [generateRandomSequence(n, True) for i in range(t)]
   
    return sequences


def implantMotifVar(k, v, t, n):
    sequences = generateRandomSequences(n, t)
    motif = generateRandomSequence(k, True)
    print("Motif : ", motif)
    
    for i in range(len(sequences)):
        L_seq = list(sequences[i])
        L_motif = list(motif)
        for nb_mutation in range(v):
            index_mutation_motif = random.randint(0,len(motif) - 1)
            L_motif[index_mutation_motif] = random.choice(nuc)
        chn_motif_mute = "".join(L_motif)
        print(chn_motif_mute)
        L_seq.insert(random.randint(0, len(L_seq) - 1), chn_motif_mute)
        
        sequences[i] = "".join(L_seq)
    return sequences


In [29]:
#generate and implant variavel motifs


sequences = implantMotifVar(k, v, t, n)
print (sequences)


Motif :  GGGCACC
GGGCACG
GGGCACG
TGGCAGC
GGGCTCG
TCGTACC
GGACACG
GGCCTCC
GGGGACA
TTGCACC
CGCCATC
GAACACC
CACAACC
GCGTACT
GGGTACG
GGCATCC
TACCACC
GGGTTAC
GGGCAGA
GGGCGCG
CGGCACC
['ATCTGCTCCGAACGTCAACTCTACACCGATCACGGGCACGACCCACGGGAGCTAGCT', 'TCCATCATTGCAAAGCCCCGATAGGGCACGACTGGTAAGTGGGTACGAATTAAGATC', 'AATGAGCTGGCAGCGTTGCTATCGGCATAGGAGCATCTTTGTGTAAGCCGGGATCCA', 'AGCCGGAACTAGCCCTCGGAAAACACGAACCCTAGGGGCTCGATTCTAGCGGGGCGG', 'GAGGACTAGCCATTCCTTTCGTACCATCTAAAACTACATATGTGGGCTCACAAATAA', 'CACTTCGACTGCTTATAGCGGACCCAAGCGGTTGCTCGGGAGGACACGCCTCTAGTC', 'ACGTGTGGCCTCCAGGGTAGAATCTTAGAGTGCTACAAGACAGGGCCCCGAATACAG', 'ACTAAAGCAGGCTCCCCCATGGGGGACACACAGGGGGCCGCGTTGTAGGACGTCCAA', 'GCTAGCCGGGGACAACGCTTGCACCACGGACCACGAACTTTCTAGCCCCAACGGCTA', 'AGCAGCTCATTTTTTATAATACCGTTATGCTCTGGTCCGAACATACGCCATCTGAGT', 'TGAAAGACCGCCTGGCAGCCGTGAGAGTTGATGCTATCGACTGAACACCACTGTCAG', 'TGGAGTGACAGCTAGGACGGCTACGCTGCGCTGTCGCATCCACAACCAAGCATTCGT', 'CCCGGAGCCCGCGTACTGGCACTCGCCAATGTAGCTTATAGAGTCTGTTAATACTCG', 'CATCAGGGGTACGTAGTGCACATGTCAC

2\. Faire une fonction pour sélectionner des positions de départ aléatoirement s = (s1, …,st).


In [30]:
#creating vector s
#Get t random positions from 0 to n - k
def generateRandomS(sequences, k):
    s = []
    for i in range(t):
        s.append(random.randint(0, len(sequences[i]) - k))
    return s

s = generateRandomS(sequences, k)
print (s)
print(len(s))

[43, 13, 25, 39, 16, 35, 27, 6, 8, 5, 44, 23, 44, 9, 18, 45, 46, 23, 50, 24]
20


3\. Extraire les motifs en utilisant le vecteur s, et construire un profile. Attention, utiliser les pseudocount 1 pour éviter les probabilités zéro.


In [31]:
#extract sequences from seqs
def extractSeqs(s, seqs):
    motifs = []
    for i in range(len(s)):
        seq_tmp = ""
        for j in range(s[i], s[i] + k):
            seq_tmp = seq_tmp + seqs[i][j]
        motifs.append(seq_tmp)        
    
    return motifs
        
motifs = extractSeqs(s, sequences)
print (motifs)

#construct Profiles
def profile(motifs, k, t):
    q = len(nuc)
    PWM = np.ones((q, k))#pseudo count
    
    L = [x for x in nuc]
    
    dictionnaire = {}
    for i in range(len(L)):
        dictionnaire[L[i]] = PWM[i]
    
    for i in range(k):
        for j in range(len(motifs)):
            dictionnaire[motifs[j][i]][i] = dictionnaire[motifs[j][i]][i] + 1
            
    #print(dictionnaire)
    for i in range(q):
        PWM[i] = dictionnaire[L[i]]
    
    #print(PWM)
            
    return dictionnaire

def profile_from_dict(dic, k):#ici k est le v de la question 14
    q = len(nuc)
    PWM = np.ones((q, k))#pseudo count
    L = [x for x in nuc]
    
    dictionnaire = {}
    for i in range(len(L)):
        dictionnaire[L[i]] = PWM[i]
            
    for cle in dic:
        for i in range(k):
            dictionnaire[cle[i]][i] = dictionnaire[cle[i]][i] + dic[cle]
    
    for i in range(q):
        PWM[i] = dictionnaire[L[i]]
    
    return dictionnaire
    

pro = profile(motifs, k, t)
print(pro)

['CACGGGA', 'AGCCCCG', 'CATAGGA', 'TCGATTC', 'TTTCGTA', 'TCGGGAG', 'GAGTGCT', 'GCAGGCT', 'GGGACAA', 'CTCATTT', 'ACACCAC', 'CGCTGCG', 'CTGTTAA', 'TACGTAG', 'CCCCGCC', 'AAAAGAA', 'GGTAGCT', 'GGGCAGA', 'CGCATCT', 'CACTGCA']
{'A': array([4., 7., 4., 8., 2., 7., 9.]), 'C': array([9., 6., 9., 6., 4., 9., 4.]), 'G': array([ 6.,  7.,  7.,  5., 12.,  4.,  5.]), 'T': array([5., 4., 4., 5., 6., 4., 6.])}


4\. Transformer le Profile en PWM. Il faut diviser chaque élément par la somme de ses colonnes.

In [32]:
#Somme des colonnes est toujours = t + len(nuc) = 24
def somme_colonnes(profile):
    colonne = []
    for i in range(k):
        s = 0
        for j in nuc:
            s = s + profile[j][i]
        colonne.append(s)
    return colonne

def PWM(profile):
    s_colonne = somme_colonnes(profile)
    profile_modifie = copy.deepcopy(profile)
    
    for i in range(k):
        for j in profile_modifie:
            profile_modifie[j][i] = profile_modifie[j][i] / s_colonne[i]
    
    return profile_modifie

pwm = PWM(pro)
print(pwm)
print(pro)

{'A': array([0.16666667, 0.29166667, 0.16666667, 0.33333333, 0.08333333,
       0.29166667, 0.375     ]), 'C': array([0.375     , 0.25      , 0.375     , 0.25      , 0.16666667,
       0.375     , 0.16666667]), 'G': array([0.25      , 0.29166667, 0.29166667, 0.20833333, 0.5       ,
       0.16666667, 0.20833333]), 'T': array([0.20833333, 0.16666667, 0.16666667, 0.20833333, 0.25      ,
       0.16666667, 0.25      ])}
{'A': array([4., 7., 4., 8., 2., 7., 9.]), 'C': array([9., 6., 9., 6., 4., 9., 4.]), 'G': array([ 6.,  7.,  7.,  5., 12.,  4.,  5.]), 'T': array([5., 4., 4., 5., 6., 4., 6.])}


5\. Faire une fonction pour calculer la probabilité d'un motif de taille k selon une PWM.


In [33]:
def probability(PWM, seq):
    """
    Hypothese : on suppose que la taille de seq est egale a la taille d'une valeur de PWM
    """
    prob = 1
    
    for i in range(len(seq)):
        prob = prob * PWM[seq[i]][i]
        
    return prob



6\. Faire une fonction pour calculer le pMostProbkmer d'une séquence, voir un exemple dans les slides de cours.

In [34]:
def tousLesKmers(ADN, k):
    """
    String -> List(String)
    Rend la liste de tous les k-mers present dans ADN
    """
    List_des_Kmer = []
    for i in range(len(ADN)-k+1):
        seq_i = ""
        for j in range(k):
            seq_i = seq_i + ADN[i+j]
        List_des_Kmer.append(seq_i)
    return List_des_Kmer

def pMostProbkmer(P, k, sequence):
    maxProb = 0
    s = 0 # position du k-mer le plus probable dans la sequence    
    L_kmer = tousLesKmers(sequence, k)
    
    for kmer in L_kmer:
        if probability(P, kmer) > maxProb:
            maxProb = probability(P, kmer)
            s = sequence.find(kmer)
    return s

s1 = pMostProbkmer(pwm, k, "GCCGGATACT")
print(s1)


0


7\. Faire une fonction pour obtenir les nouvelles positions de départ s = (s1, …,st), c’était à dire les positions qui contiens les k-mer le plus probables.

In [35]:
def getNewS(P, k, sequences):
    
    s = []
    for sequence in sequences:
        s.append(pMostProbkmer(P, k, sequence))
    
    return s

ns = getNewS(pwm, k, sequences)
print (ns)

[23, 15, 8, 1, 43, 6, 45, 35, 26, 0, 38, 6, 2, 13, 18, 40, 20, 26, 9, 24]


8\. La condition d’arrêt d’algorithme est le non changement du Profile d'une itération à l'autre, faire une fonction pour comparer deux Profils (matrice) et détecter le changement.

In [36]:
def changeProfile(P1, P2):
    
    changement = False
    
    L = [x for x in nuc]
    L2 = [["Pareil" for i in range(len(P1[L[0]]))] for j in range(len(P1))]
    dictionnaire = {}
    
    for i in range(len(L)):
        dictionnaire[L[i]] = L2[i]
        
    for nucleo in nuc:
        for i in range(len(P1[nucleo])):
            if P1[nucleo][i] != P2[nucleo][i]:
                dictionnaire[nucleo][i] = P2[nucleo][i]
                changement = True #Il y a un changement

    for definition in dictionnaire:
        print(definition, " ", dictionnaire[definition])
    
    
    return changement 
    

def getScore(P, k):
    sc = 0
    S = random.choice(nuc)
    
    for i in range(len(P[S])):
        colonne = []
        for cle in P:
            colonne.append(P[cle][i])
        sc = sc + max(colonne)
    
    return sc

def score(s, seqs):
    sc = 0
    profile1 = profile(extractSeqs(s, seqs), k, t)
    
    L_max_colonne = []
    
    for i in range(len(profile1["A"])):
        L_tmp = []
        for j in nuc:
            L_tmp.append(profile1[j][i])
        sc = sc + max(L_tmp)
    
    return (sc)

print(score(s, sequences))

ns = getNewS(pwm, k, sequences)
print (s, ns)

sc = getScore(pro, k)
print (sc)
#test = changeProfile(pwm, pwm)
#print(test)

63.0
[43, 13, 25, 39, 16, 35, 27, 6, 8, 5, 44, 23, 44, 9, 18, 45, 46, 23, 50, 24] [23, 15, 8, 1, 43, 6, 45, 35, 26, 0, 38, 6, 2, 13, 18, 40, 20, 26, 9, 24]
63.0


9\. Implementer l'algorithme ``GreedyProfileMotifSearch`` en utilisant les fonctions precedentes. 


In [37]:
def GreedyProfileMotifSearch(sequences, t, n, k):
    
    bestScore = 0
    s = generateRandomS(sequences, k)
    
    while score(s, sequences) > bestScore:
        pro = profile(extractSeqs(s, sequences), k, t)
        pwm = PWM(pro)
        bestScore = score(s, sequences)
        #print(bestScore)
            
        s = getNewS(pwm, k, sequences)
   
    return s, bestScore
    
        
s, bestScore = GreedyProfileMotifSearch(sequences, t, n, k)   
motifs = extractSeqs(s, sequences)
print (motifs)
        

['ACGGGCA', 'ACTGGTA', 'GCTGGCA', 'ACTAGCC', 'ACTAGCC', 'ACACGCC', 'ACAGGGC', 'ACAGGGG', 'GCTAGCC', 'TCTGGTC', 'CCTGGCA', 'GCTAGGA', 'ACTGGCA', 'ACTGGCA', 'CCTAGCC', 'TCTGGCG', 'ACTGGCC', 'ACTGGGG', 'ATAGGCC', 'GCAAGCA']


10\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

Reponse: Trouvé en exécutant beaucoup de fois...

11\. Vous avez certainement observer que l’algorithme ne produire pas toujours la même sortie et que les résultats dépendent de la sélection aléatoire des positions de départ s. Pour augmenter nous chances de retrouvez les bons motifs, nous allons implémenter une version itérative ``GreedyProfileMotifSearchIte`` 
qui a chaque fois sauvegarde s et le score du profile associé à s, après I itération, l’algorithme renvoie le vecteur s ayant le plus grand score.

In [38]:
def maximumIndice(liste):
    maxi = liste[0]
    indice_max = 0
    longueur=len(liste)
    for i in range(longueur):
        if liste[i] >= maxi:
            maxi = liste[i]
            indice_max = i
    return indice_max

def GreedyProfileMotifSearchIte(sequences, t, n, k, It):
    Scores = []
    Positions = []
    
    for i in range(It):
        s, score = GreedyProfileMotifSearch(sequences, t, n, k)
        Scores.append(score)
        Positions.append(s)
    
    J = maximumIndice(Scores)
    
    return Positions[J], Scores[J]
    
s, bestScore = GreedyProfileMotifSearchIte(sequences, t, n, k, 100)
motifs = extractSeqs(s, sequences)
print (motifs)

['ACGGGCA', 'ACTGGTA', 'GCTGGCA', 'ACTAGCC', 'ACTAGCC', 'ACACGCC', 'ACGTGTG', 'GCAGGCT', 'GCTTGCA', 'ACATACG', 'ACTGTCA', 'GCTGTCG', 'ACTGGCA', 'ACTGGCA', 'AATGGCA', 'ACTTGCT', 'ACTGGCC', 'AATGGCG', 'GCCGGCG', 'ACTGACG']


12\. Tester algorithme  ``GreedyProfileMotifSearchIte`` sur vos données de chipSeq.

In [39]:
k=7; v=1; t=20; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_8.fasta" #path of your fasta seqs


sequencesChip   = readFasta(genome, n)
t = len(sequencesChip)
print (sequencesChip[8], t, n, k)
print(len(sequencesChip[len(sequencesChip) - 1]))

s, bestScore = GreedyProfileMotifSearchIte(sequencesChip, t, n, k, 100)
print (bestScore, s)
motifs = extractSeqs(s, sequencesChip)
print (motifs)

ACCTTATGGTATTTATTTTCAAGATTCAGCTACACCTAATCAAGAAGGTATTTTAGAATTACATGATAATATTATGTTTT 912 80 7
12
4279.0 [19, 16, 37, 34, 11, 0, 17, 44, 67, 8, 30, 12, 72, 20, 20, 20, 68, 35, 54, 16, 63, 60, 2, 68, 30, 42, 48, 17, 64, 26, 27, 45, 60, 71, 1, 46, 17, 49, 0, 33, 71, 27, 19, 42, 54, 50, 55, 8, 64, 1, 4, 25, 55, 49, 19, 47, 44, 22, 46, 48, 2, 1, 47, 7, 14, 2, 21, 16, 50, 6, 9, 37, 71, 3, 69, 60, 65, 23, 22, 26, 68, 65, 27, 50, 30, 3, 18, 33, 64, 26, 45, 53, 0, 63, 2, 47, 47, 27, 15, 63, 1, 68, 34, 31, 20, 24, 62, 48, 24, 73, 34, 26, 50, 6, 72, 73, 26, 32, 56, 14, 46, 67, 35, 28, 20, 8, 22, 29, 70, 60, 66, 47, 57, 1, 7, 43, 50, 13, 68, 2, 44, 45, 40, 15, 1, 22, 5, 6, 27, 41, 57, 66, 56, 27, 32, 1, 57, 30, 32, 0, 55, 12, 69, 67, 7, 30, 59, 14, 70, 59, 70, 1, 39, 31, 24, 41, 7, 50, 5, 70, 47, 11, 64, 26, 55, 5, 17, 19, 57, 70, 50, 49, 60, 25, 62, 63, 65, 58, 0, 48, 29, 24, 39, 62, 6, 68, 3, 51, 73, 35, 49, 31, 49, 47, 29, 34, 16, 29, 12, 64, 59, 37, 43, 41, 17, 47, 37, 55, 41, 63, 44, 56, 66, 9, 6

13\. Nous allons implémenter l'algorithme ``randomProjection``. Faire une fonction pour générer une projection de l à k, voir un exemple dans les slides de cours

In [53]:
k=7 #taille de motif
v=4 #nb de positions variable dans le motif
t=20 #nb de sequences
n=50 #longuer des sequence

def getRandomFixePositions(k, n):
    listR = []
       
    for i in range(n):
        posHasard = random.randint(0, k-1)
        while posHasard in listR:
            posHasard = random.randint(0, k-1)
        listR.append(posHasard)
    return listR

lR = getRandomFixePositions(7, 4)
lR.sort()
print (lR)

def generateKey(positions, motif):
    m = ""
    for i in positions:
        m = m + motif[i]
    
    return m


def tousLesKmersList(seqs, k):
    """
    List(String) -> List(String)
    Rend la liste de tous les k-mers present dans les sequences de seqs
    """
    List_des_Kmer = []
    for sequence in seqs:
        for i in range(len(sequence) - k + 1):
            seq_i = ""
            for j in range(k):
                seq_i = seq_i + sequence[i+j]
            List_des_Kmer.append(seq_i)
            
    return List_des_Kmer

def removeLowComplexe(motifs, minrep):
    validMotif = []
    for mot in motifs:
        if mot.count('A') < minrep and mot.count('C') < minrep and mot.count('G') < minrep and mot.count('T') < minrep:
            validMotif.append(mot)
    return validMotif


def removeLowComplexePair(motifs):
    validMotif = []
    
    for cle in motifs:
        list_tmp = []
        list_tmp.append(cle[0])
        for i in range(2, len(cle), 2):
            if cle[i] == list_tmp[0]:
                list_tmp.append(cle[i])
        cpt_pos_pair = len(list_tmp)
        list_tmp = []
        list_tmp.append(cle[1])
        for i in range(3, len(cle), 2):
            if cle[i] == list_tmp[0]:
                list_tmp.append(cle[i])
        cpt_pos_impair = len(list_tmp)
        
        if not cpt_pos_pair == ((len(cle) // 2) + 1) and not cpt_pos_impair == (len(cle) // 2):
            validMotif.append(cle)
              
    return validMotif


minrep = 5

[0, 3, 4, 5]


14\. Implémenter l'algorithme ``randomProjection``.

In [82]:
def randomProjection(k, v, sequences):
    motifs  = {}; motifsSeq  = {}
    L_L_tuple = removeLowComplexe(tousLesKmersList(sequences, k), minrep)
    L_L_tuple = removeLowComplexePair(L_L_tuple) #On enleve les motifs peu complexes
    rfp = getRandomFixePositions(k, v)
    
    for mot in L_L_tuple:
        m = generateKey(rfp, mot)
        if m in motifs: 
            motifs[m] = motifs[m] + 1
        else:
            motifs[m] = 1
        if m in motifsSeq:
            motifsSeq[m].append(mot)
        else:
            motifsSeq[m] = [mot]
    
    return motifs, motifsSeq
#motifsSort = sorted(motifs, reverse=True, key=motifs.get)
motifs, motifsSeq = randomProjection(7, 4, sequences)
print (motifs)
#print(motifsSeq)
p_test = profile_from_dict(motifs, 4)
print(p_test)


{'TAGT': 3, 'CTCC': 5, 'CCTT': 2, 'GTCG': 8, 'AGCC': 8, 'ACGT': 4, 'CTAC': 6, 'GCAC': 5, 'TCCG': 4, 'CGGA': 5, 'AATA': 2, 'AACC': 12, 'CCAG': 6, 'TGAT': 3, 'TCTA': 4, 'AACA': 4, 'CATC': 4, 'ACAT': 4, 'CCAT': 3, 'GTCA': 4, 'TCGA': 7, 'CAAC': 6, 'ACTC': 3, 'CCCG': 4, 'GGAA': 3, 'GACT': 5, 'GTGC': 6, 'CCGA': 8, 'AAGC': 5, 'GGAG': 9, 'AGCG': 12, 'CGGC': 6, 'CCAA': 4, 'CACC': 4, 'ACCG': 4, 'CGCA': 3, 'GCCC': 8, 'GCGC': 7, 'ACGA': 8, 'GAGC': 5, 'TGGG': 4, 'GGTA': 5, 'CAAG': 3, 'TGGC': 4, 'ATTC': 3, 'TCCC': 4, 'TCAA': 5, 'GATT': 7, 'CTTC': 4, 'AACT': 1, 'ATAT': 2, 'GTAG': 4, 'CGAC': 4, 'CACA': 3, 'GACG': 4, 'TCGC': 4, 'ACAC': 3, 'GCTC': 5, 'GCAG': 3, 'GGGA': 2, 'CAGT': 4, 'ATGA': 6, 'CACG': 7, 'GACC': 3, 'GCTG': 3, 'TGGA': 6, 'ACTT': 4, 'TGAG': 4, 'GGGT': 2, 'GTTA': 5, 'GAGA': 6, 'AGGT': 6, 'CTTG': 2, 'TTAA': 4, 'TAAC': 6, 'ACTG': 2, 'AGTA': 2, 'GAAA': 6, 'AAAT': 4, 'TTGT': 2, 'CTAA': 7, 'CAAA': 4, 'CAGG': 5, 'AGGC': 7, 'GCCT': 9, 'CTAG': 10, 'GGGG': 3, 'TGCC': 6, 'GATG': 5, 'CGTC': 4, 'TCGG'

15\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

reponse: Sans itérations, le resultat depend GRANDEMENT de generateRandomS(sequences, k)

16\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [85]:
def randomProjIt(sequences, k, v, It):
    total_score=[]
    total_seqsMotifs=[]
    total_score_dict = {} #dictionnaire des occurences
    
    for i in range(It):
        score, seqsMotif = randomProjection(k, v, sequences)
        
        p_score = profile_from_dict(score, v)
        score_t = getScore(p_score, v)
        
        for cle in score:
            if cle in total_score_dict:
                total_score_dict[cle] = total_score_dict[cle] + score[cle]
            else:
                total_score_dict[cle] = score[cle]
        
        total_score.append(score_t)
        total_seqsMotifs.append(seqsMotif)
    
    J = maximumIndice(total_score)
        
    return total_score[J], total_seqsMotifs[J], total_score_dict

    #ou alors ...(ce que j'avais fait au debut...)
    """
    total_score={}
    total_seqsMotifs={}
    
    for i in range(It):
        score, seqsMotif = randomProjection(k, v, sequences)
        
        for cle in score:
            if cle in total_score:
                total_score[cle] = total_score[cle] + score[cle]
            else:
                total_score[cle] = score[cle]
            if cle in total_seqsMotifs:
                for mot in seqsMotif[cle]:
                    if mot not in total_seqsMotifs[cle]:
                        total_seqsMotifs[cle].append(mot)
            else:
                total_seqsMotifs[cle] = seqsMotif[cle]
    return total_score, total_seqsMotifs
    """
        

score, seqsMotif, total_score_dict = randomProjIt(sequences, 7, 4, 100)

print (score)
print(total_score_dict)
#print(seqsMotif)

1012.0
{'TCCA': 371, 'CTTT': 210, 'CCGC': 459, 'GCCT': 387, 'AGTG': 362, 'AACC': 438, 'CACT': 387, 'GCGC': 458, 'TGAC': 361, 'CTAG': 407, 'ACCA': 474, 'AAGA': 239, 'CATC': 385, 'TCCG': 321, 'CTAT': 333, 'TCAC': 374, 'ATCA': 321, 'CATA': 347, 'ACCC': 495, 'CATT': 291, 'CCAC': 439, 'AGAA': 260, 'TACC': 406, 'CTCA': 407, 'ACGC': 552, 'CAAC': 451, 'GCTG': 381, 'GGCA': 441, 'GGAT': 349, 'CGCC': 447, 'ACGA': 433, 'CAGC': 559, 'GCGG': 452, 'AGCG': 489, 'CAAG': 438, 'CCCC': 192, 'CCGA': 523, 'ACAC': 509, 'CACG': 524, 'GGCC': 493, 'GGAC': 467, 'AGCC': 556, 'GAGA': 369, 'TCGG': 344, 'ATAG': 291, 'GAGG': 472, 'CGCA': 496, 'TCTG': 269, 'ACCT': 390, 'TAAC': 349, 'TTTC': 180, 'GTCA': 400, 'CGAT': 418, 'ACTC': 391, 'AATA': 229, 'AAGT': 320, 'GACT': 362, 'CGAG': 441, 'CCAA': 463, 'GCCA': 513, 'ATCC': 375, 'GAGC': 421, 'GGTG': 351, 'CGAA': 450, 'ACGT': 421, 'CAGA': 453, 'TCCC': 261, 'GTGA': 388, 'TGCG': 404, 'ATTA': 252, 'AAGC': 451, 'GAGT': 344, 'TGTG': 226, 'GTAG': 420, 'GGGA': 424, 'ATGG': 372, 'CAG

17\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. Puis générér le LOGO du motif trouvé

In [99]:
k=7; v=1; t=len(sequencesChip); n=80

score, seqsMotif, total_score_dict = randomProjIt(sequencesChip, 7, 4, 100)
print(score)
total_score_dict = sorted(total_score_dict.items(), reverse = True) #donne une liste de tuple ordonné
total_score_dict = sorted(total_score_dict,key = lambda x : x[1] , reverse = True) #liste triée selon les valeur de cles
print(total_score_dict)
#print(seqsMotif)

def element_des_lists_valeur(dic):
    """
    Dict -> List[String]
    Rend la liste des motifs des valeur de chaque cle de dic
    """
    L = []
    for cle in dic:
        for mot in dic[cle]:
            L.append(mot)
    
    return L

def concensus(P,k):
    concen = ""
    L_tmp = []
    
    for cle in P:
        for i in range(len(nuc)):
            L_tmp.append(P[cle])
    
    for i in range(k):
        maxi_tmp = []
        for j in range(len(L_tmp)):
            maxi_tmp.append(L_tmp[j][i])
        for cle in P:
            if P[cle][i] == max(maxi_tmp):
                concen = concen + cle
                break
    return concen
 
#test perso...  
m1, n1 = total_score_dict[0]

p_t = profile(seqsMotif[m1], 7, t)
conc = concensus(p_t, 7)
print(conc)
"""
test_L_val_cle = element_des_lists_valeur(seqsMotif)
test_profil = profile(test_L_val_cle, 7, t)
print(test_profil)
conc = concensus(test_profil, 7)
print(conc)
"""


71196.0
[('TATA', 50409), ('TTAA', 50348), ('AATT', 49534), ('ATAT', 49446), ('ATTA', 49336), ('TAAT', 49313), ('TTTA', 44805), ('TATT', 44046), ('TTAT', 43830), ('ATTT', 43810), ('TAAA', 40218), ('AAAT', 39989), ('AATA', 39599), ('ATAA', 39530), ('GTAT', 35318), ('TTGA', 35012), ('GATT', 34614), ('ATTG', 34560), ('TATG', 34302), ('GTTA', 34281), ('TTCA', 34096), ('AGTT', 34089), ('CATT', 33846), ('ATTC', 33797), ('GAAT', 33790), ('TATC', 33747), ('TGTA', 33681), ('TGAT', 33665), ('CTAT', 33565), ('AATG', 33544), ('TAGT', 33530), ('TTAG', 33487), ('TGAA', 33362), ('GTAA', 33320), ('ACTT', 33281), ('TAGA', 33271), ('ATGT', 33211), ('ATGA', 32875), ('TAAG', 32826), ('TCTA', 32752), ('TACT', 32739), ('ATCT', 32717), ('AATC', 32709), ('TCAT', 32596), ('ATAG', 32593), ('GATA', 32477), ('CTTA', 32453), ('CAAT', 32395), ('AGAT', 32354), ('AAGT', 32343), ('AGTA', 32144), ('TTAC', 32110), ('TACA', 32008), ('CTAA', 31978), ('ATAC', 31962), ('ATCA', 31435), ('ACAT', 31340), ('TCAA', 31329), ('TAA

'\ntest_L_val_cle = element_des_lists_valeur(seqsMotif)\ntest_profil = profile(test_L_val_cle, 7, t)\nprint(test_profil)\nconc = concensus(test_profil, 7)\nprint(conc)\n'